In [1]:
import netgen.gui
from ngsolve import *
from ngsolve.internal import visoptions
from netgen.meshing import ImportMesh

optfile ./ng.opt does not exist - using default values
togl-version : 2
OCC module loaded
loading ngsolve library
NGSolve-6.2.2304
Using Lapack
Including sparse direct solver UMFPACK
Running parallel using 10 thread(s)


2024-08-08 20:57:34.646 Python[12284:23400068] CGSSetSurfaceColorSpace failed - error 1000 (windowID:254441)
2024-08-08 20:57:34.646 Python[12284:23400068] CGSSetSurfaceColorSpace failed - error 1000 (windowID:254441)


In [2]:
ngmesh = ImportMesh('cell_mesh_R1_opt.mesh')

In [5]:
from netgen.meshing import FaceDescriptor
isinplane = lambda el: all((ngmesh[v][2] < -27 for v in el.vertices))
fd_id = ngmesh.Add(FaceDescriptor(surfnr=2,domin=1,domout=0,bc=2))
for el in ngmesh.Elements2D():
    if isinplane(el):
        el.index = fd_id
ngmesh.SetBCName(fd_id-1, "new_bc")

In [6]:
mesh = Mesh(ngmesh)
Draw(mesh)

In [7]:
fes = H1(mesh, order = 1, dirichlet="new_bc")
u,v = fes.TnT()

In [8]:
a = BilinearForm(fes, symmetric=True)
a += grad(u).Trace() * grad(v).Trace()*ds
a.Assemble()

# external force
force = sin(x)
f = LinearForm(fes)
f += force*v*ds
f.Assemble()

Homogeneous Dirichlet data

In [9]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs())*f.vec
Draw(gfu, mesh, "u", autoscale=False)

Inhomogeneous Dirichlet data

In [10]:
gfu = GridFunction(fes)
gfu.Set(-sin(x), definedon=mesh.Boundaries("new_bc"))
r = f.vec.CreateVector()
r.data = f.vec - a.mat*gfu.vec
gfu.vec.data += a.mat.Inverse(fes.FreeDofs()) * r
Draw(gfu, mesh, "u", autoscale=False)